In [4]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import os
from os import mkdir
from os.path import isdir, isfile

import warnings
warnings.filterwarnings("ignore")

!pip install ipython-autotime 
%load_ext autotime
#для создания датасета
!pip install pymupdf 
import fitz 

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.86 s (started: 2023-01-25 17:44:23 +03:00)


In [5]:
URL_TEMPLATE = "https://amulex.ru/docs/"
docs = requests.get(URL_TEMPLATE, headers={'User-Agent': 'Custom'})
print(docs.status_code)

200
time: 297 ms (started: 2023-01-25 17:44:30 +03:00)


In [25]:
'''
Функция make_dir скачивает pdf-файлы с сайта https://amulex.ru и сохраняет их по папкам, разделяя на классы

На вход подается ссылка на категорию договоров (category_contract_url), название класса договора (folder_name)
1. Переходит в категорию category_contract_url
2. Проходя по всей категории, скачивает договоры, переименовывает их с использованием _
3. Сохраняет скаченные файлы в папку с нужным классом

Параметры:
1. category_contract_url:
    Ссылка на категорию договоров на сайте. Договоры определенного класса могут быть из разных категорий
2. folder_name:
    Название класса договора из списка: Договоры аренды, Договоры купли-продажи, Договоры оказания услуг, Договоры подряда, Договоры поставки
'''
def make_dir(category_contract_url, folder_name, folder_path='C:/Users/265375/Desktop/Work/parsing/'):
    category_cite = requests.get('https://amulex.ru' + category_contract_url, headers={'User-Agent': 'Custom'})
    categ = bs(category_cite.text, "html.parser")
    contract_url = categ.find_all('li',class_='items__item', id = False)
    contract_names = categ.find_all('h5',class_="items__item-title")
    for i in range(len(contract_url)):
        name_words = contract_names[i].text.split()
        name = ''
        for k in name_words:
            name += ''.join(e for e in k if e.isalnum())
            if k != name_words[-1]:
                name += '_'
        contract_cite = requests.get('https://amulex.ru' + contract_url[i].a['href'], headers={'User-Agent': 'Custom'})
        pars = bs(contract_cite.text, "html.parser")
        contract_download_url = pars.find('div',class_='download__dropdown js-dropdown')
        x = requests.get(contract_download_url.a['href'])
        if isfile(folder_path + folder_name + '/' + name + '.pdf') == False:
            with open(folder_path + folder_name + '/' + name + '.pdf', 'wb') as f:
                f.write(x.content)

time: 0 ns (started: 2023-01-25 17:56:43 +03:00)


In [27]:
soup = bs(docs.text, "html.parser")
category_contract_url = soup.find_all('a', class_='category__item-link')
folder_names = ['Договоры аренды', 'Договоры купли-продажи', 'Договоры оказания услуг', 'Договоры подряда', 'Договоры поставки']
for i in folder_names:
    mkdir('C:/Users/265375/Desktop/Work/parsing/' + str(i))
for j in range(len(category_contract_url)):
    s=category_contract_url[j].h2.text
    re.sub(r"\s+", "", s)
    if 'аренд' in s:
        make_dir(category_contract_url[j]['href'], folder_names[0])
    elif 'купл' in s:
        make_dir(category_contract_url[j]['href'], folder_names[1])
    elif 'услуг' in s:
        make_dir(category_contract_url[j]['href'], folder_names[2])
    elif 'подряд' in s:
        make_dir(category_contract_url[j]['href'], folder_names[3])
    elif 'постав' in s:
        make_dir(category_contract_url[j]['href'], folder_names[4])

time: 55min 19s (started: 2023-01-25 17:56:53 +03:00)


In [28]:
path = 'C:/Users/265375/Desktop/Work/parsing/'
docs = pd.DataFrame(columns = ['text','filename','target'])
count = 0

for fold_name in folder_names:
    folder_path = os.chdir('C:/Users/265375/Desktop/Work/parsing/' + fold_name)
    for name in os.listdir(folder_path):
        doc = fitz.open(name)
        text = ''
        for current_page in range(doc.page_count):
            text += doc.get_page_text(current_page,"text")
        text = '\n'.join(text.split('\n')[::2])
        new_row = {'text': text,
                   'filename': name,
                   'target': fold_name}
        docs = docs.append(new_row, ignore_index=True)

time: 21.5 s (started: 2023-01-25 19:03:40 +03:00)


In [29]:
docs.to_csv('C:/Users/265375/Desktop/Work/parsing/parsed_data.csv', sep='\t',index=False, encoding = 'utf-8')

time: 453 ms (started: 2023-01-25 19:04:01 +03:00)
